In [1]:
cd ../../..

/home/users/cdonoso/astromer/ASTROMER


# Using ASTROMER as layer's model

In [2]:
import tensorflow as tf
import pandas as pd
import os

from tensorflow.keras.layers import BatchNormalization, Dense, LSTM, LayerNormalization
from tensorflow.keras import Input, Model

from presentation.experiments.clf.classifiers import build_mlp_att, build_lstm_att
from core.data import load_dataset, inference_pipeline, pretraining_pipeline
from core.astromer import ASTROMER
from core.training.scheduler import CustomSchedule

os.environ["CUDA_VISIBLE_DEVICES"]='1'
%load_ext autoreload
%autoreload 2

First we load ASTROMER pre-trained weights

In [3]:
batch_size = 512
astromer = ASTROMER()
astromer.load_weights('./runs/macho/')

2022-04-04 10:46:57.612121: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-04 10:46:58.029450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22354 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6


First we load data

In [4]:
batch_size = 512
max_obs    = 200
datapath   = './data/records/alcock/fold_0/alcock/'

In [5]:
catalog = pd.read_csv('{}/objects.csv'.format(datapath))
n_classes = catalog.shape[0]
catalog

,label,size
0,Cep_0,1082
1,Cep_1,583
2,EC,6724
3,LPV,2946
4,RRab,7297
5,RRc,1662


In [6]:
traindata = load_dataset(os.path.join(datapath, 'train'), repeat=1, shuffle=True)
valdata   = load_dataset(os.path.join(datapath, 'val'), repeat=1)
testdata  = load_dataset(os.path.join(datapath, 'test'), repeat=1)

[INFO] Shuffling


## Classification

When using `inference_pipeline()` the output is the classification labels instead of the reconstruction

In [7]:
train_batches = inference_pipeline(traindata,
                                   batch_size=batch_size,
                                   max_obs=max_obs,
                                   n_classes=n_classes,
                                   shuffle=False)
val_batches   = inference_pipeline(valdata,
                                   batch_size=batch_size,
                                   max_obs=max_obs,
                                   n_classes=n_classes,
                                   shuffle=False)

[INFO] Inference mode. Cutting 200-len windows
[INFO] Inference mode. Cutting 200-len windows


Now we use the encoder layer from the pretrained ASTROMER instance

In [8]:
encoder = astromer.get_layer('encoder')

and creates the classifier model

In [9]:
serie  = Input(shape=(max_obs, 1), batch_size=None, name='input')
times  = Input(shape=(max_obs, 1), batch_size=None, name='times')
mask   = Input(shape=(max_obs, 1), batch_size=None, name='mask')
placeholder = {'input':serie, 'mask_in':mask, 'times':times}

mask_rnn = tf.cast(1.-placeholder['mask_in'][...,0], dtype=tf.bool)

x = encoder(placeholder, training=False)
x = LSTM(256, dropout=.3, return_sequences=True)(x, mask=mask_rnn)
x = LayerNormalization()(x)
x = LSTM(256, dropout=.3)(x, mask=mask_rnn)
x = LayerNormalization()(x)
x = Dense(n_classes)(x)

classifier = Model(inputs=placeholder, outputs=x, name="LSTMATT")

we compile our model with `adam` optimizer and `CategoricalCrossentropy` loss

In [15]:
from tensorflow.keras.losses import CategoricalCrossentropy

classifier.compile(optimizer='adam',
                   loss=CategoricalCrossentropy(from_logits=True),
                   metrics='accuracy')

and let's train!

In [ ]:
history = classifier.fit(train_batches,
                         epochs=5,
                         validation_data=val_batches)

Epoch 1/5


2022-04-04 10:49:15.575241: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-04-04 10:49:16.504696: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


119/119 [==============================] - 42s 280ms/step - loss: 1.2561 - accuracy: 0.5463 - val_loss: 0.8550 - val_accuracy: 0.6866
Epoch 2/5
119/119 [==============================] - 32s 266ms/step - loss: 0.8911 - accuracy: 0.6711 - val_loss: 0.8503 - val_accuracy: 0.6857
Epoch 3/5
 52/119 [============>.................] - ETA: 15s - loss: 0.8899 - accuracy: 0.6607